<h1 style="color:#b57c01;">[Pie Chart] 財政部出口統計</h1>

In [ ]:
#pip install --upgrade nbformat


In [ ]:
import pandas as pd
import os
import plotly.express as px

from dotenv import load_dotenv

input_file ="mof_export_statistic.csv"


load_dotenv()


data_path = os.environ.get("PROCESSED_DATA_DIR")

fPath = os.path.join ( data_path, input_file ) 
df = pd.read_csv ( fPath )
df= df.fillna(0)


In [ ]:
cols_name = df.columns
cols_name

Index(['國家/地區別', '總計', '1.活動物；動物產品', '(1)水產品', '2.植物產品', '(1)食用蔬菜',
       '(2)食用果實及堅果', '3.動植物油脂', '4.調製食品；飲料及菸酒', '(1)調製食品', '(2)飲料、酒類及醋',
       '(3)菸及菸葉代用品', '5.礦產品', '(1)石油煉製品', '6.化學品', '(1)有機化學品', '7.塑膠、橡膠及其製品',
       '(1)塑膠原料', '(2)塑膠製品', '(3)橡膠及其製品', '8.毛皮及其製品', '9.木及木製品',
       '10.紙漿；紙及其製品；印刷品', '11.紡織品', '(1)紗布', '12.鞋、帽及其他飾品', '13.非金屬礦物製品',
       '14.珠寶及貴金屬製品', '15.基本金屬及其製品', '(1)鋼鐵及其製品', '(2)其他基本金屬及其製品',
       '16.機械及電機設備', '(1)電子零組件', '(2)機械', '(3)電機產品', '(4)資通與視聽產品', '(5)家用電器',
       '17.運輸工具', '18.光學及精密儀器；鐘錶；樂器', '(1)光學及精密儀器', '19.其他', '(1)家具',
       '(2)玩具與運動用品', '月', '年'],
      dtype='object')

In [ ]:
CLstr = "國家/地區別"
df[CLstr].unique()

array(['亞洲', '中國大陸', '香港', '日本', '南韓', '新南向18國(註1)', '東協10國', '泰國',
       '馬來西亞', '印尼', '菲律賓', '新加坡', '越南', '汶萊', '緬甸', '柬埔寨', '寮國', '南亞',
       '印度', '巴基斯坦', '孟加拉', '斯里蘭卡', '尼泊爾', '不丹', '哈薩克', '中東', '伊朗', '伊拉克',
       '以色列', '科威特', '阿曼', '卡達', '沙烏地阿拉伯', '土耳其', '阿拉伯聯合大公國', '北美洲', '美國',
       '加拿大', '中美洲', '墨西哥', '南美洲', '巴西', '智利', '秘魯', '歐洲', '德國', '法國',
       '荷蘭', '英國', '俄羅斯', '奧地利', '比利時', '匈牙利', '義大利', '波蘭', '西班牙', '瑞典',
       '瑞士', '烏克蘭', '歐盟(註2)', '東歐', '非洲', '阿爾及利亞', '安哥拉', '埃及', '南非',
       '肯亞', '奈及利亞', '大洋洲', '澳大利亞', '巴布亞紐幾內亞', '紐西蘭'], dtype=object)

In [ ]:
#import plotly.io as 機械及電機設備pio
#pio.renderers.default = "browser" # 設定渲染器為瀏覽器
# 

colName = "總計" #"16.機械及電機設備" 
df = df[~df[ CLstr ].isin( ["北美洲","亞洲", "新南向18國(註1)", "東協10國", "中美洲" , "歐盟(註2)"] ) ]


In [ ]:
import pandas as pd
import plotly.express as px

# Settings
topN = 10
CLstr = "國家/地區別"
colName = "總計"

# 1️⃣ Aggregate by year per country
df_yearly = df.groupby(['年', CLstr], as_index=False)[colName].sum()

# 2️⃣ Rank Top N per year
df_yearly['rank'] = df_yearly.groupby('年')[colName].rank(method='first', ascending=False)
df_topN = df_yearly[df_yearly['rank'] <= topN].copy()

# 3️⃣ Sort for plotting
df_topN = df_topN.sort_values(['年', colName], ascending=[True, False])
df_topN["年"] = df_topN["年"].astype(str)  # make sure frame names are string

# 4️⃣ Create figure
fig = px.bar(
    df_topN,
    x=colName,
    y=CLstr,
    color=CLstr,
    orientation='h',
    animation_frame="年",
    animation_group=CLstr,
    color_discrete_sequence=px.colors.qualitative.Pastel,
    width=900,
    height=600,
    range_x=[0, df_topN[colName].max() * 1.1],  # dynamic x-axis max
)

# 5️⃣ Update each frame for dynamic y-axis ordering and text labels
for frame in fig.frames:
    year_data = df_topN[df_topN["年"] == frame.name].sort_values(colName, ascending=True)
    
    # Update y-axis order
    frame.layout.yaxis = dict(
        categoryorder="array",
        categoryarray=year_data[CLstr].tolist()#,
        #autorange='reversed'  # top bar at top
    )
    
    
# 6️⃣ Layout
fig.update_layout(
    title=f"年度國家總計 Top {topN} Bar Race",
    xaxis_title=colName,
    yaxis_title=CLstr,
)

fig.update_layout(
    yaxis=dict(
        categoryorder="array",
        categoryarray=[]#,
        #autorange='reversed'  # <-- this flips the bars so top bar is at top
    )
)

# 7️⃣ Slow down animation
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 2500  # 2.5 sec per frame

# 8️⃣ Show figure
fig.show()





In [ ]:
df_topN[df_topN["年"] == 114].sort_values(colName, ascending=False)

,年,國家/地區別,總計,rank


In [ ]:
df_topN[ [ "國家/地區別" , "年" ,"總計" , "rank" ] ]
df_topN.head(120)

NameError: name 'df_sorted' is not defined

In [ ]:
df_selected = df[ df[CLstr]=="日本" ]
df_selected = df.groupby(['年', CLstr], as_index=False)[colName].sum()

df_selected [ [ "國家/地區別" , "年" , "總計"] ]

,國家/地區別,年,總計
0,不丹,104,158
1,中國大陸,104,73270118
2,中東,104,6969850
3,以色列,104,610811
4,伊拉克,104,42386
...,...,...,...
721,阿曼,114,264553
722,阿爾及利亞,114,99459
723,非洲,114,1486605
724,香港,114,70392820
